In [1]:
import os
import sys
import json
import torch
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from transformers import BertModel, BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I0326 13:17:17.971240 139831343572800 file_utils.py:39] PyTorch version 1.1.0 available.


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
from fgc_support_retri.ser_extractor import *
from fgc_support_retri.utils import *
from fgc_support_retri import config
from evaluation.fgc_eval import *
from evaluation.eval import eval_sp_fgc, eval_fgc_atype

In [4]:
model_folder = config.TRAINED_MODELS / "20200326_sgroup_lr=2e-5" 

In [5]:
extractor = Sgroup_extractor(model_folder)

I0326 13:17:26.928295 139831343572800 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0326 13:17:27.828396 139831343572800 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0326 13:17:27.831607 139831343572800 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

In [21]:
eval_data = json_load(config.FGC_TRAIN)

In [22]:
get_answer_sp(eval_data, force=True)

100%|██████████| 882/882 [00:00<00:00, 27544.38it/s]

{'text': '電視名人。為川普集團前任董事長兼總裁', 'text_cn': '电视名人。为川普集团前任董事长兼总裁', 'start': 76, 'end': 94}
{'text': '川普娛樂公司的創辦人，在全世界經營房地產', 'text_cn': '川普娱乐公司的创办人，在全世界经营房地产', 'start': 95, 'end': 115}
{'text': '熱，電，輻射', 'text_cn': '热，电，辐射', 'start': 22, 'end': 28}
{'text': '', 'text_cn': '', 'start': 0, 'end': 0}
{'text': '校園注意教室之空氣流通，落實勤洗手，有呼吸道症狀時應配戴口罩；打噴嚏時應用面紙', 'text_cn': '校园注意教室之空气流通，落实勤洗手，有呼吸道症狀时应配戴口罩；打喷嚏时应用面纸', 'start': 88, 'end': 127}
{'text': '他人交談時，儘可能保持1公尺以上之距離，避免病毒傳播。', 'text_cn': '他人交谈时，尽可能保持1公尺以上之距离，避免病毒传播。', 'start': 143, 'end': 170}
{'text': '', 'text_cn': '', 'start': 0, 'end': 0}


In [ ]:
all_sp_predictions = []
all_atype_predictions = []
all_items = []
for d in tqdm(eval_data):
    for q in d['QUESTIONS']:
        sp_preds, _, sp_scores = extractor.predict(q, d)
        q['sp'] = sp_preds
        q['sp_scores'] = sp_scores

  3%|▎         | 27/882 [00:22<14:10,  1.01it/s]

In [ ]:
all_items, all_sp_predictions, all_answer_sp = eval_from_threshold(eval_data, threshold=0.5)

In [ ]:
metrics = eval_sp_fgc(all_items, all_sp_predictions)

In [ ]:
metrics = eval_sp_fgc(all_answer_sp, all_sp_predictions)

# draw intervals

In [ ]:
def get_interval_span():
    intervals = []
    last = 0
    step = 10
    for i in range(step):
        now = round(last+1/step, 1)
        intervals.append((last, now))
        last = now
    return intervals

In [ ]:
def get_interval(score):
    # get interval span
    intervals = []
    last = 0
    step = 10
    for i in range(step):
        now = round(last+1/step, 1)
        intervals.append((last, now))
        last = now
    
    # get interv_i
    for interv_i, interv in enumerate(intervals):
        if interv[0] <= score < interv[1]:
            return interv_i

In [ ]:
def createLabels(data):
    all_height = 0
    for item in data:
        all_height += item.get_height()
    for item in data:
        height = item.get_height()
        plt.text(
            item.get_x()+item.get_width()/2., 
            height*1.05, 
            '{}% ({})'.format(int(height/all_height*100), int(height)),
            ha = "center",
            va = "bottom",
        )

In [ ]:
intervals = get_interval_span()

In [ ]:
interv_scores_p = [0] * 10
interv_scores_n = [0] * 10
interv_scores_answer = [0] * 10
interv_scores_not_answer = [0] * 10
for document in eval_data:
    for question in document['QUESTIONS']:
        if not question['SHINT']:
            continue
        for sent_i, sp_score in enumerate(question['sp_scores']):
            if sent_i in question['SHINT']:
                interv_scores_p[get_interval(sp_score)] += 1
                if sent_i in question['answer_sp']:
                    interv_scores_answer[get_interval(sp_score)] += 1
                else:
                    interv_scores_not_answer[get_interval(sp_score)] += 1
            else:
                interv_scores_n[get_interval(sp_score)] += 1

In [ ]:
plt.figure(figsize=(30, 6))
plt.subplot(131)
A = plt.bar([str(interv[1]) for interv in intervals], interv_scores_p)
createLabels(A)
plt.title('all sp')

In [ ]:
plt.figure(figsize=(30, 6))
plt.subplot(131)
B = plt.bar([str(interv[1]) for interv in intervals], interv_scores_n)
createLabels(B)
plt.title('not sp')

In [ ]:
plt.figure(figsize=(30, 6))
plt.subplot(131)
B = plt.bar([str(interv[1]) for interv in intervals], interv_scores_answer)
createLabels(B)
plt.title('answer sp')

In [ ]:
plt.figure(figsize=(30, 6))
plt.subplot(131)
B = plt.bar([str(interv[1]) for interv in intervals], interv_scores_not_answer)
createLabels(B)
plt.title('not answer sp')